In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

DATA CLEAN

In [ ]:
df = pd.read_csv("../dataset/games-features.csv")
if(df.isnull().values.any()):
    df.dropna(inplace=True)#null degerler silindi
df=df.drop(['QueryID',
         'DeveloperCount',
         'PackageCount','SteamSpyOwnersVariance','SteamSpyPlayersEstimate','SteamSpyPlayersVariance',
         'PublisherCount','AchievementHighlightedCount','PriceCurrency','Background',
         'DetailedDescrip','DRMNotice','ExtUserAcctNotice','HeaderImage','LegalNotice'],axis=1)
df = df.drop("QueryName", axis=1)
df = df.rename(columns={"ResponseName": "AppName"})
df = df.drop("ResponseID", axis=1)
df = df.rename(columns={"ResponseID": "AppID"})
df = df.rename(columns={"MovieCount": "GameTrailerCount"})
df = df.rename(columns={"SteamSpyOwners": "UserCount"})
df['Email']=df.SupportEmail==" "
df = df.drop("SupportEmail", axis=1)
df['Websites']=df.Website==" "
df = df.drop("Website", axis=1)
df.info()

In [ ]:
print("Toplam oyun sayısı : {0}".format(len(df.AppName.unique())))
dfNonZeroMetacritic = df[df.Metacritic != 0]
print("Ortalama metacritix : {0}".format(dfNonZeroMetacritic.Metacritic.mean()))
dfNonPrice = df[df.PriceFinal != 0]
print("Ortalama fiyat $ : {0}".format(dfNonZeroMetacritic.PriceFinal.mean()))

DATA2 CLEAN

In [ ]:
df200kPlayer= pd.read_csv("../dataset/steam-200k.csv",names=['UserID', 'Game', 'Action', 'Hours', 'Other'])
df200kPlayer=df200kPlayer [df200kPlayer .Action != 'purchase']# satın alma verisi silindi
df200kPlayer.info()
df200kPlayer.isnull().values.any()
if(df.isnull().values.any()):
    df.dropna(inplace=True)#null degerler silindi

In [ ]:
print("Toplam oyun sayısı : {0}".format(len(df200kPlayer.Game.unique())))
print("Toplam kullanıcı sayısı: {0}".format(len(df200kPlayer.UserID.unique())))

DATA VISUALIZATION

In [ ]:
figsize=(12,8)

In [ ]:
Genres_names=['GenreIsAction','GenreIsIndie','GenreIsAdventure',
'GenreIsCasual',
'GenreIsStrategy',
'GenreIsRPG',
'GenreIsSimulation',
'GenreIsEarlyAccess',
'GenreIsFreeToPlay',
'GenreIsRacing',
'GenreIsSports',
'GenreIsMassivelyMultiplayer',
'GenreIsNonGame']

In [ ]:
GenreIsAction=df.index[df['GenreIsAction'] == True].tolist()
GenreIsIndie = df.index[df['GenreIsIndie'] == True].tolist()
GenreIsAdventure=df.index[df['GenreIsAdventure'] == True].tolist()
GenreIsCasual=df.index[df['GenreIsCasual'] == True].tolist()
GenreIsStrategy=df.index[df['GenreIsStrategy'] == True].tolist()
GenreIsRPG=df.index[df['GenreIsRPG'] == True].tolist()
GenreIsSimulation=df.index[df['GenreIsSimulation'] == True].tolist()
GenreIsEarlyAccess=df.index[df['GenreIsEarlyAccess'] == True].tolist()
GenreIsFreeToPlay=df.index[df['GenreIsFreeToPlay'] == True].tolist()
GenreIsRacing=df.index[df['GenreIsRacing'] == True].tolist()
GenreIsSports=df.index[df['GenreIsSports'] == True].tolist()
GenreIsMassivelyMultiplayer=df.index[df['GenreIsMassivelyMultiplayer'] == True].tolist()
GenreIsNonGame=df.index[df['GenreIsNonGame'] == True].tolist()

Genres=[GenreIsAction,GenreIsIndie,GenreIsAdventure,
GenreIsCasual,
GenreIsStrategy,
GenreIsRPG,
GenreIsSimulation,
GenreIsEarlyAccess,
GenreIsFreeToPlay,
GenreIsRacing,
GenreIsSports,
GenreIsMassivelyMultiplayer,
GenreIsNonGame]

In [ ]:
meta=[]
ort=[]
oyuncu=[]
oyuncu_ort=[]
fiyat_ort=[]
fiyat=[]
for i in range(len(Genres)):
    for j in Genres[i]:
    
        if df['Metacritic'][j] >0 and df['PriceInitial'][j] > 0:
            
            meta.append(df['Metacritic'][j])
            oyuncu.append(df['UserCount'][j])
            fiyat.append((df['PriceInitial'][j]))
    ort.append(np.mean(meta))
    oyuncu_ort.append(np.mean(oyuncu))
    fiyat_ort.append(np.mean(fiyat))

In [ ]:
ort_df=pd.DataFrame(list(zip(oyuncu_ort, ort ,fiyat_ort)),
              columns=['oyuncu sayısı','metacritic','fiyat' ], index=Genres_names)

In [ ]:
ort_df.drop(['oyuncu sayısı'], axis=1).plot(kind='bar', legend=False, title="Mean price and metacritic Score by Genre",figsize=figsize)

In [ ]:
ort_df.drop(['oyuncu sayısı','metacritic'], axis=1).plot(kind='bar', legend=False, title="Mean Price  by Genre",figsize=figsize)

In [ ]:
ort_df.drop(['oyuncu sayısı','fiyat'], axis=1).plot(kind='bar', legend=False, title="Mean Metacritic Score by Genre",figsize=figsize)

In [ ]:
total_user_game = df200kPlayer.groupby('Game')['UserID'].agg('count').sort_values(ascending=False)[0:20] #oyun ismine göre kaç kullanıcı oldugunu gruplandı
total_user_game

In [ ]:
total_hours = df200kPlayer.groupby('Game')['Hours'].agg(np.sum).sort_values(ascending=False)[0:20]#oynama saatleri
total_hours

In [ ]:
total_user_game = pd.DataFrame({'game': total_user_game.index, 'total user game': total_user_game.values})
total_hours = pd.DataFrame({'game': total_hours.index, 'total time': total_hours.values})
fig, ax =plt.subplots(1,2,figsize=(12,15))
sns.barplot(y = 'game', x = 'total user game', data = total_user_game, ax=ax[0])
sns.barplot(y = 'game', x = 'total time', data = total_hours, ax=ax[1])
ax[0].set(xlabel='Oyuncu sayısı', ylabel='Oyun ismi')
ax[1].set(xlabel='Toplam oynanan saat', ylabel='Oyun ismi')
ax[1].yaxis.tick_right()
ax[1].yaxis.set_label_position("right")
for i in range(0,2):
    ax[i].tick_params(axis='y', labelsize=18)
    ax[i].xaxis.label.set_size(20)

In [ ]:
fig, ax =plt.subplots(1,2,figsize=figsize)
df.groupby(['PlatformLinux','PlatformWindows','PlatformMac'])['AppName'].size().plot(kind='bar',ax=ax[0])
df.groupby(['CategorySinglePlayer','CategoryMultiplayer'])['AppName'].size().plot(kind='bar',ax=ax[1])
ax[0].set(ylabel='Oyun ismi')
ax[1].set(ylabel='Oyun ismi')

PREDICT

In [ ]:
data = df.copy()
data=data.select_dtypes(exclude=['object'])
data = data[data.Metacritic != 0]
data = data[data.IsFree != True]
y=data["PriceFinal"]
X=data.drop(["PriceFinal","PriceInitial"],axis=1)
X=pd.DataFrame(X)
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.15,random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score,classification_report
rf_params={"max_depth":[2,5,8,10,12],
          "max_features":[2,5,8,10,12,15],
          "n_estimators": [10,50,200,500],
          "min_samples_split":[5,10,20,50]}
rf_model=RandomForestRegressor()

rf_cv_model= GridSearchCV(rf_model,
                         rf_params,
                         cv=10,
                         n_jobs=-1,
                         verbose=5)
rf_cv_model.fit(X_train,y_train)

In [ ]:
print("best params"+str(rf_cv_model.best_params_))

In [ ]:
rf_tuned=RandomForestRegressor(max_depth=12,
                               max_features=8,
                               min_samples_split=5,
                               n_estimators=200)
rf_tuned.fit(X_train,y_train)

In [ ]:
y_pred_test =rf_tuned.predict(X_test)
y_pred_train =rf_tuned.predict(X_train)
rf_tuned.score(X_test, y_test)

In [ ]:
from sklearn.metrics import mean_squared_error 
mean_squared_error(y_test,y_pred_test)

In [ ]:
mean_squared_error(y_train,y_pred_train) 

In [ ]:
importances = rf_tuned.feature_importances_
indices = np.argsort(importances)[-6:]
indices
#en basariyi tahminler
columns_list=[]
columns_list=df.columns
columns_list[indices]

In [ ]:
ig, axs = plt.subplots(2, 3)
axs[0, 0].scatter(df.PriceFinal, df.DLCCount)
axs[0, 0].set_title('Axis [0,0]')
axs[0, 1].scatter(df.PriceFinal, df.DemoCount)
axs[0, 1].set_title('Axis [0,1]')
axs[0, 2].scatter(df.PriceFinal, df.ScreenshotCount)
axs[0, 2].set_title('Axis [0,2]')
axs[1, 0].scatter(df.PriceFinal, df.Metacritic)
axs[1, 0].set_title('Axis [1,0]')
axs[1, 1].scatter(df.PriceFinal, df.RequiredAge)
axs[1, 1].set_title('Axis [2,0]')
axs[1, 2].scatter(df.PriceFinal, df.RecommendationCount)
axs[1, 2].set_title('Axis [2,0]')

for ax in axs.flat:
    ax.set(xlabel='x-label', ylabel='y-label')

# Hide x labels and tick labels for top plots and y ticks for right plots.
for ax in axs.flat:
    ax.label_outer()

In [ ]:
for i in range(100):

    print(rf_tuned.predict([X_test.iloc[i]]))
    print(y_test.iloc[i])